In [3]:
from dataclasses import dataclass
from math import gcd, ceil
import re
import numpy as np
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools import cache
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest, batched, pairwise
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx

from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, sortdict, data, quantify, atom, atoms, list_atoms, list_multiply, mapt, Node
from aocutils.grid import TURNRIGHT, TURNLEFT, move, gridneigh, arr_to_dict, grid_to_dict, neighbors, complexneighbors, arr_neighbors, iterate, Dim, dimensions, positive, ranges2grid, manhattan, conv1d, conv2d, rotate, rot90
from aocutils.maze import graphparse, bfs, dijkstra, distances, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst, shoelace
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree, Trie
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot, animate
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard
E = (0,1); W = (0,-1); N = (-1,0); S = (1,0)
winds = [N, E, S, W]
inf = float('inf')

In [4]:
f = open("in.txt").read().split("\n")
lines = [line for line in f]
lines[0]

'This is day16 -999  '

In [82]:
# grid
grid, neigh = gridneigh("t0.txt", to_dict=True, parser=lambda x: [(ch) for ch in x])
(minr, minc), (maxr, maxc), (rows, cols) = dimensions(grid)

start = ([k for k,v in grid.items() if v == 'S'].pop(), E)
goal = [k for k,v in grid.items() if v == 'E'].pop()
goals = {(goal, w) for w in winds}

In [83]:
def getneigh(loc):
    coor, wind = loc
    turn = [((coor, w), 1000) for w in winds if w != wind]
    newloc = move(coor, wind)
    if newloc in grid and grid[newloc] != '#':
        turn.append(((newloc, wind), 1))
    return turn
getneigh(((5,5), (0,1)))

[(((5, 5), (-1, 0)), 1000),
 (((5, 5), (1, 0)), 1000),
 (((5, 5), (0, -1)), 1000),
 (((5, 6), (0, 1)), 1)]

In [88]:
#| export
import heapq
def dijkstra(connections,start, goal=None):
    """
    Requires a dict with as values a LIST of tuples (neighbor, weight)
    Or a function returning a list of tuples with neighbors and weights per node

    Returns
    if goal == None:    return all paths from start
    elif goal found:    returns path to goal
    else:               returns False
    """
    seen = set() # the locations that have been explored
    frontier = [(0,start)] # the locations that still need to be visited
    isfunction = callable(connections)
    parents = {start: (None,0)}

    def get_path(parents,goal):
        cur = goal
        path = [cur]
        cost = parents[cur][1]
        while cur != start:
            cur = parents[cur][0]
            path.append(cur)
        path.reverse()
        return path,cost

    while frontier:
        # print('\n\n',frontier,'\n',parents)
        search_cost, search_node = heapq.heappop(frontier)
        # print('looking for', search_node,search_cost)
        if search_node == goal: break
        if isfunction: neighbors = connections(search_node)
        else: neighbors = connections.get(search_node,None)
        if neighbors:
            for n in neighbors:
                # print('n',n)
                if n[0] not in parents or n[1]+ search_cost < parents[n[0]][1]:
                    # print('updating')
                    heapq.heappush(frontier,(search_cost+n[1],n[0]))
                    # paths[n] = paths[search_node]+[n]
                    parents[n[0]]= (search_node,search_cost+n[1])
                        # return paths[goal],parents
        seen.add(search_node)
    if goal is None: return parents
    else:
        for g in goals:
            if g in parents: 
                return parents
                return get_path(parents,g)
    return False



In [90]:
parents = dijkstra(getneigh, start, goals)
parents


{((13, 1), (0, 1)): (None, 0),
 ((13, 1), (-1, 0)): (((13, 1), (0, 1)), 1000),
 ((13, 1), (1, 0)): (((13, 1), (0, 1)), 1000),
 ((13, 1), (0, -1)): (((13, 1), (0, 1)), 1000),
 ((13, 2), (0, 1)): (((13, 1), (0, 1)), 1),
 ((13, 2), (-1, 0)): (((13, 2), (0, 1)), 1001),
 ((13, 2), (1, 0)): (((13, 2), (0, 1)), 1001),
 ((13, 2), (0, -1)): (((13, 2), (0, 1)), 1001),
 ((13, 3), (0, 1)): (((13, 2), (0, 1)), 2),
 ((13, 3), (-1, 0)): (((13, 3), (0, 1)), 1002),
 ((13, 3), (1, 0)): (((13, 3), (0, 1)), 1002),
 ((13, 3), (0, -1)): (((13, 3), (0, 1)), 1002),
 ((12, 1), (-1, 0)): (((13, 1), (-1, 0)), 1001),
 ((12, 1), (0, 1)): (((12, 1), (-1, 0)), 2001),
 ((12, 1), (1, 0)): (((12, 1), (-1, 0)), 2001),
 ((12, 1), (0, -1)): (((12, 1), (-1, 0)), 2001),
 ((11, 1), (-1, 0)): (((12, 1), (-1, 0)), 1002),
 ((11, 1), (0, 1)): (((11, 1), (-1, 0)), 2002),
 ((11, 1), (1, 0)): (((11, 1), (-1, 0)), 2002),
 ((11, 1), (0, -1)): (((11, 1), (-1, 0)), 2002),
 ((10, 1), (-1, 0)): (((11, 1), (-1, 0)), 1003),
 ((10, 1), (0, 

In [85]:
bestlocs, bestscore = dijkstra(getneigh, start, goals)
locs = {loc for loc, wind in bestlocs}
for loc in {loc for loc, wind in bestlocs}:
    grid[loc] = '#'
    a = dijkstra(getneigh, start, goals)
    if a != False:
        print(a)
        newlocs, newscore = a
        if newscore == bestscore:
            locs |= {loc for loc, wind in newlocs}
    grid[loc] = '.'
len(locs)
    
    
    

([((13, 1), (0, 1)), ((13, 1), (-1, 0)), ((12, 1), (-1, 0)), ((11, 1), (-1, 0)), ((11, 1), (0, 1)), ((11, 2), (0, 1)), ((11, 3), (0, 1)), ((11, 4), (0, 1)), ((11, 5), (0, 1)), ((11, 5), (-1, 0)), ((10, 5), (-1, 0)), ((9, 5), (-1, 0)), ((8, 5), (-1, 0)), ((7, 5), (-1, 0)), ((7, 5), (0, 1)), ((7, 6), (0, 1)), ((7, 7), (0, 1)), ((7, 8), (0, 1)), ((7, 9), (0, 1)), ((7, 9), (-1, 0)), ((6, 9), (-1, 0)), ((5, 9), (-1, 0)), ((5, 9), (0, 1)), ((5, 10), (0, 1)), ((5, 11), (0, 1)), ((5, 11), (-1, 0)), ((4, 11), (-1, 0)), ((3, 11), (-1, 0)), ((3, 11), (0, -1)), ((3, 10), (0, -1)), ((3, 9), (0, -1)), ((3, 9), (-1, 0)), ((2, 9), (-1, 0)), ((1, 9), (-1, 0)), ((1, 9), (0, 1)), ((1, 10), (0, 1)), ((1, 11), (0, 1)), ((1, 12), (0, 1)), ((1, 13), (0, 1)), ((1, 13), (1, 0))], 11028)
([((13, 1), (0, 1)), ((13, 1), (-1, 0)), ((12, 1), (-1, 0)), ((11, 1), (-1, 0)), ((11, 1), (0, 1)), ((11, 2), (0, 1)), ((11, 3), (0, 1)), ((11, 4), (0, 1)), ((11, 5), (0, 1)), ((11, 5), (-1, 0)), ((10, 5), (-1, 0)), ((9, 5), (-

44

In [87]:
sorted(locs)

[(1, 13),
 (2, 13),
 (3, 13),
 (4, 13),
 (5, 13),
 (6, 13),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 7),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 11),
 (7, 13),
 (8, 3),
 (8, 5),
 (8, 11),
 (8, 13),
 (9, 1),
 (9, 2),
 (9, 3),
 (9, 5),
 (9, 11),
 (9, 13),
 (10, 1),
 (10, 5),
 (10, 11),
 (10, 13),
 (11, 1),
 (11, 2),
 (11, 3),
 (11, 4),
 (11, 5),
 (11, 11),
 (11, 13),
 (12, 1),
 (12, 11),
 (12, 13),
 (13, 1),
 (13, 11),
 (13, 12),
 (13, 13)]

In [50]:
98416

98416

In [51]:
len(res[0])-1

60

In [ ]:
516

In [ ]:
# graph
conn = graphparse("in.txt", parser=lambda line: [x for x in line.split()])
conn = graphparse(
    "in.txt",
    parser=lambda line: [int(x) if x.isnumeric() else x for x in line.split()],
    weights=True,
)

In [7]:
E = 1; W = -1; N = 1j; S = -1j
TURNLEFT = lambda x: x * 1j
TURNRIGHT = lambda x: x * -1j

# parse grid with complex numbers
f = open("in.txt").read().split("\n")
lines = [list(line.split()) for line in f]
grid = arr_to_dict(lines, tocomplex=True)
complexneighbors(3+2j)

{(2+2j), (3+1j), (3+3j), (4+2j)}